# 第3天：RAG（检索增强生成）核心技术

## 今日学习目标
1. 深入理解RAG的工作原理和架构
2. 掌握向量数据库的使用（Chroma、FAISS）
3. 实现文档嵌入和相似度检索
4. 构建完整的RAG问答系统
5. 学习检索质量优化技巧
6. 实现混合搜索（语义+关键词）

In [ ]:
# 环境设置
import os
import shutil  # 添加用于目录操作
from dotenv import load_dotenv
import numpy as np
from typing import List, Dict, Any

load_dotenv()
os.environ["LANGCHAIN_TRACING_V2"] = "false"

print("✅ 环境配置完成")

## 1. RAG架构深度解析

### RAG解决的核心问题
1. **知识时效性**：LLM训练数据有截止时间，无法获取最新信息
2. **领域专业性**：通用模型在特定领域知识上可能不够深入
3. **幻觉问题**：模型可能生成看似合理但实际错误的信息
4. **可解释性**：难以追溯回答的信息来源

### RAG工作流程
```
用户查询 → 查询向量化 → 向量相似度搜索 → 检索相关文档 → 构建增强提示词 → LLM生成回答
```

In [ ]:
# 核心导入
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import Chroma, FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.schema import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers import BM25Retriever, EnsembleRetriever
import chromadb
from chromadb.config import Settings

# 初始化核心组件
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
embeddings = OpenAIEmbeddings()

print("🤖 LLM和嵌入模型初始化完成")

## 2. 创建知识库：文档处理和嵌入

### 2.1 准备示例文档数据

In [ ]:
# 创建AI技术知识库文档
ai_knowledge_base = [
    {
        "title": "人工智能基础",
        "content": """
        人工智能（Artificial Intelligence, AI）是计算机科学的一个分支，旨在创建能够执行通常需要人类智能的任务的机器和软件。
        AI的主要目标包括学习、推理、感知、语言理解和问题解决。现代AI主要基于机器学习技术，特别是深度学习。
        AI应用广泛，包括自然语言处理、计算机视觉、语音识别、推荐系统、自动驾驶等领域。
        """,
        "category": "基础概念",
        "keywords": ["人工智能", "AI", "机器学习", "深度学习"]
    },
    {
        "title": "机器学习算法",
        "content": """
        机器学习是人工智能的核心技术，主要分为三类：监督学习、无监督学习和强化学习。
        监督学习包括分类和回归任务，常用算法有线性回归、逻辑回归、决策树、随机森林、支持向量机（SVM）等。
        无监督学习主要用于发现数据中的隐藏模式，包括聚类、降维和关联规则挖掘。
        强化学习通过与环境交互学习最优策略，在游戏AI、机器人控制等领域应用广泛。
        """,
        "category": "技术详解",
        "keywords": ["机器学习", "监督学习", "无监督学习", "强化学习", "算法"]
    },
    {
        "title": "深度学习与神经网络",
        "content": """
        深度学习是机器学习的一个子集，使用多层神经网络来建模和理解复杂的数据模式。
        卷积神经网络（CNN）主要用于图像处理，在计算机视觉任务中表现卓越。
        循环神经网络（RNN）和长短期记忆网络（LSTM）适合处理序列数据，如文本和时间序列。
        Transformer架构彻底改变了自然语言处理领域，GPT、BERT等大型语言模型都基于此架构。
        """,
        "category": "技术详解",
        "keywords": ["深度学习", "神经网络", "CNN", "RNN", "LSTM", "Transformer"]
    },
    {
        "title": "自然语言处理技术",
        "content": """
        自然语言处理（NLP）是AI的重要分支，致力于让计算机理解和生成人类语言。
        传统NLP技术包括分词、词性标注、命名实体识别、句法分析等基础任务。
        现代NLP主要基于深度学习，特别是Transformer架构的预训练语言模型。
        主要应用包括机器翻译、文本摘要、情感分析、问答系统、对话系统等。
        BERT、GPT、T5等预训练模型在各种NLP任务上都取得了突破性进展。
        """,
        "category": "应用领域",
        "keywords": ["自然语言处理", "NLP", "BERT", "GPT", "机器翻译", "文本摘要"]
    },
    {
        "title": "计算机视觉基础",
        "content": """
        计算机视觉是AI的另一个重要领域，目标是让机器能够理解和解释视觉信息。
        基础任务包括图像分类、目标检测、图像分割、人脸识别等。
        卷积神经网络（CNN）是计算机视觉的核心技术，经典架构包括LeNet、AlexNet、VGG、ResNet等。
        现代计算机视觉还涉及生成对抗网络（GAN）、扩散模型等生成技术。
        应用领域包括自动驾驶、医学影像分析、工业质检、增强现实等。
        """,
        "category": "应用领域",
        "keywords": ["计算机视觉", "CNN", "图像分类", "目标检测", "GAN"]
    },
    {
        "title": "大语言模型发展",
        "content": """
        大语言模型（Large Language Models, LLMs）是近年来AI领域的重大突破。
        从2018年的BERT到2019年的GPT-2，再到2020年的GPT-3，模型规模不断增长。
        ChatGPT和GPT-4的发布标志着AI进入了新的时代，展现了强大的对话和推理能力。
        这些模型基于Transformer架构，通过大规模预训练获得了广泛的知识和能力。
        LLMs的应用包括文本生成、代码编写、翻译、摘要、问答等多个方面。
        """,
        "category": "前沿技术",
        "keywords": ["大语言模型", "LLM", "GPT", "ChatGPT", "BERT", "Transformer"]
    },
    {
        "title": "AI伦理与安全",
        "content": """
        随着AI技术的快速发展，AI伦理和安全问题日益重要。
        主要关注点包括算法偏见、隐私保护、透明度、可解释性、责任归属等。
        AI安全涉及对抗性攻击、模型鲁棒性、数据安全等技术问题。
        需要建立完善的AI治理框架，确保AI技术的负责任发展和应用。
        国际组织和各国政府正在制定相关的AI伦理准则和法律法规。
        """,
        "category": "伦理安全",
        "keywords": ["AI伦理", "AI安全", "算法偏见", "隐私保护", "可解释性"]
    }
]

# 转换为Document对象
documents = []
for item in ai_knowledge_base:
    doc = Document(
        page_content=item["content"].strip(),
        metadata={
            "title": item["title"],
            "category": item["category"],
            "keywords": item["keywords"]
        }
    )
    documents.append(doc)

print(f"📚 创建了 {len(documents)} 篇AI技术文档")
print(f"📄 示例文档: {documents[0].metadata['title']}")
print(f"📝 内容预览: {documents[0].page_content[:100]}...")

### 2.2 文档分块处理

In [ ]:
# 创建文本分割器
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,  # 每块最大字符数
    chunk_overlap=50,  # 重叠字符数，保持上下文连贯性
    separators=["\n\n", "\n", "。", "！", "？", ";", ",", " ", ""]  # 分割优先级
)

# 分割文档
splits = text_splitter.split_documents(documents)

print(f"📊 文档分块统计:")
print(f"原始文档数: {len(documents)}")
print(f"分割后块数: {len(splits)}")
print(f"平均每文档块数: {len(splits) / len(documents):.1f}")

# 显示分块示例
print(f"\n📝 分块示例:")
for i, chunk in enumerate(splits[:3]):
    print(f"\n块 {i+1} (长度: {len(chunk.page_content)} 字符):")
    print(f"来源: {chunk.metadata['title']}")
    print(f"内容: {chunk.page_content[:150]}...")

## 3. 向量数据库实践

### 3.1 Chroma向量数据库

In [ ]:
# 创建Chroma向量数据库
print("🗄️ 创建Chroma向量数据库...")

# 设置持久化目录
persist_directory = "./chroma_db"

# 清除可能存在的数据库（确保干净的开始）
import shutil
if os.path.exists(persist_directory):
    shutil.rmtree(persist_directory)
    print(f"🗑️ 清理旧的数据库目录: {persist_directory}")

# 创建目录（如果不存在）
os.makedirs(persist_directory, exist_ok=True)
print(f"📁 创建数据库目录: {persist_directory}")

# 创建向量存储
chroma_vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=embeddings,
    persist_directory=persist_directory,
    collection_name="ai_knowledge"
)

print(f"✅ Chroma数据库创建完成")
print(f"📊 存储了 {chroma_vectorstore._collection.count()} 个向量")

# 测试相似度搜索
test_query = "什么是深度学习？"
similar_docs = chroma_vectorstore.similarity_search(test_query, k=3)

print(f"\n🔍 相似度搜索测试")
print(f"查询: {test_query}")
print(f"找到 {len(similar_docs)} 个相似文档:")

for i, doc in enumerate(similar_docs, 1):
    print(f"\n{i}. {doc.metadata['title']}")
    print(f"   内容: {doc.page_content[:120]}...")

### 3.2 FAISS向量数据库（高性能）

In [ ]:
# 创建FAISS向量数据库
print("\n⚡ 创建FAISS向量数据库...")

faiss_vectorstore = FAISS.from_documents(
    documents=splits,
    embedding=embeddings
)

print(f"✅ FAISS数据库创建完成")

# 保存到本地
faiss_vectorstore.save_local("./faiss_index")
print(f"💾 FAISS索引已保存到本地")

# 测试FAISS搜索性能
import time

# 对比搜索速度
queries = [
    "机器学习有哪些类型？",
    "CNN在计算机视觉中的应用",
    "GPT模型的发展历程"
]

print(f"\n⏱️ 搜索性能对比:")

for query in queries:
    # Chroma搜索
    start_time = time.time()
    chroma_results = chroma_vectorstore.similarity_search(query, k=2)
    chroma_time = time.time() - start_time
    
    # FAISS搜索
    start_time = time.time()
    faiss_results = faiss_vectorstore.similarity_search(query, k=2)
    faiss_time = time.time() - start_time
    
    print(f"\n查询: {query}")
    print(f"Chroma: {chroma_time:.4f}s | FAISS: {faiss_time:.4f}s")
    print(f"FAISS加速比: {chroma_time/faiss_time:.1f}x")

### 3.3 相似度搜索与评分

In [ ]:
# 带评分的相似度搜索
def analyze_similarity_search(vectorstore, query: str, k: int = 5):
    """分析相似度搜索结果"""
    print(f"\n🎯 详细相似度分析")
    print(f"查询: {query}")
    print(f"检索数量: {k}")
    
    # 获取带评分的结果
    results = vectorstore.similarity_search_with_score(query, k=k)
    
    print(f"\n📋 搜索结果:")
    for i, (doc, score) in enumerate(results, 1):
        print(f"\n{i}. 相似度得分: {score:.4f}")
        print(f"   标题: {doc.metadata['title']}")
        print(f"   分类: {doc.metadata['category']}")
        print(f"   内容: {doc.page_content[:150]}...")
        
        # 分析相关性
        if score < 0.3:
            relevance = "🟢 高度相关"
        elif score < 0.5:
            relevance = "🟡 中度相关"
        else:
            relevance = "🔴 低度相关"
        print(f"   相关性: {relevance}")

# 测试不同类型的查询
test_queries = [
    "深度学习和神经网络的关系",  # 精确匹配
    "AI在医疗领域的应用",        # 间接相关
    "如何学习编程？"            # 不相关
]

for query in test_queries:
    analyze_similarity_search(chroma_vectorstore, query, k=3)
    print("-" * 60)

## 4. 构建完整的RAG系统

### 4.1 基础RAG实现

In [ ]:
# 创建RAG提示词模板
rag_prompt = ChatPromptTemplate.from_template("""
你是一个专业的AI技术专家，请基于以下参考文档回答用户的问题。

参考文档:
{context}

用户问题: {question}

请遵循以下要求：
1. 基于参考文档提供准确的回答
2. 如果参考文档中没有相关信息，请明确说明
3. 可以适当扩展解释，但要确保准确性
4. 用清晰、专业的语言回答

回答:
""")

# 创建检索器
retriever = chroma_vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}  # 检索top-3最相似的文档
)

# 格式化文档函数
def format_docs(docs):
    """格式化检索到的文档"""
    formatted = []
    for i, doc in enumerate(docs, 1):
        formatted.append(f"文档{i}: {doc.metadata['title']}\n{doc.page_content}")
    return "\n\n".join(formatted)

# 创建RAG链
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)

print("🔗 RAG链创建完成")

# 测试RAG系统
test_questions = [
    "什么是深度学习？它与机器学习有什么关系？",
    "Transformer架构在NLP中有什么重要作用？",
    "计算机视觉的主要应用领域有哪些？",
    "大语言模型的发展历程是怎样的？"
]

print("\n🤖 RAG系统测试:")

for i, question in enumerate(test_questions, 1):
    print(f"\n--- 问题 {i} ---")
    print(f"❓ {question}")
    
    # 获取答案
    answer = rag_chain.invoke(question)
    print(f"\n🤖 RAG回答:")
    print(answer)
    
    # 显示检索到的文档
    retrieved_docs = retriever.get_relevant_documents(question)
    print(f"\n📚 使用的参考文档:")
    for j, doc in enumerate(retrieved_docs, 1):
        print(f"{j}. {doc.metadata['title']} ({doc.metadata['category']})")
    
    print("-" * 80)

### 4.2 高级RAG：带引用和置信度

In [ ]:
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser

# 定义结构化输出模型
class RAGResponse(BaseModel):
    """RAG回答结构化输出"""
    answer: str = Field(description="基于参考文档的答案")
    confidence: float = Field(description="答案置信度 (0-1)", ge=0, le=1)
    sources: List[str] = Field(description="参考文档标题列表")
    has_sufficient_info: bool = Field(description="参考文档是否包含足够信息")
    additional_notes: str = Field(description="补充说明或限制条件")

# 创建高级RAG提示词
advanced_rag_prompt = ChatPromptTemplate.from_template("""
你是一个AI技术专家，请基于参考文档回答问题，并提供结构化输出。

参考文档:
{context}

用户问题: {question}

请分析参考文档的相关性，并评估你能提供准确答案的置信度。

{format_instructions}
""")

# 创建解析器
rag_parser = PydanticOutputParser(pydantic_object=RAGResponse)

# 创建高级RAG链
advanced_rag_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough(),
        "format_instructions": lambda _: rag_parser.get_format_instructions()
    }
    | advanced_rag_prompt
    | llm
    | rag_parser
)

print("\n🚀 高级RAG系统测试:")

advanced_questions = [
    "CNN和RNN在深度学习中的区别是什么？",
    "AI伦理方面有哪些主要关注点？",
    "量子计算对AI发展有什么影响？"  # 知识库中可能没有的信息
]

for i, question in enumerate(advanced_questions, 1):
    print(f"\n--- 高级分析 {i} ---")
    print(f"❓ {question}")
    
    try:
        response = advanced_rag_chain.invoke(question)
        
        print(f"\n📊 分析结果:")
        print(f"置信度: {response.confidence:.2f}")
        print(f"信息充分性: {'✅ 充分' if response.has_sufficient_info else '❌ 不充分'}")
        print(f"参考来源: {', '.join(response.sources)}")
        
        print(f"\n🤖 回答:")
        print(response.answer)
        
        if response.additional_notes:
            print(f"\n📝 补充说明:")
            print(response.additional_notes)
            
    except Exception as e:
        print(f"解析错误: {e}")
    
    print("-" * 80)

## 5. 检索质量优化

### 5.1 混合搜索（语义搜索 + 关键词搜索）

In [ ]:
# 创建BM25检索器（关键词搜索）
print("🔍 创建混合搜索系统...")

# 从文档创建BM25检索器
bm25_retriever = BM25Retriever.from_documents(splits)
bm25_retriever.k = 3

# 创建语义搜索检索器
semantic_retriever = chroma_vectorstore.as_retriever(
    search_kwargs={"k": 3}
)

# 创建集成检索器（混合搜索）
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, semantic_retriever],
    weights=[0.3, 0.7]  # BM25权重30%，语义搜索权重70%
)

print("✅ 混合搜索系统创建完成")

# 对比不同检索方法
def compare_retrieval_methods(query: str):
    """对比不同检索方法的效果"""
    print(f"\n🔍 检索方法对比")
    print(f"查询: {query}")
    
    # BM25检索
    bm25_docs = bm25_retriever.get_relevant_documents(query)
    print(f"\n📋 BM25检索结果:")
    for i, doc in enumerate(bm25_docs, 1):
        print(f"{i}. {doc.metadata['title']}")
    
    # 语义检索
    semantic_docs = semantic_retriever.get_relevant_documents(query)
    print(f"\n🎯 语义检索结果:")
    for i, doc in enumerate(semantic_docs, 1):
        print(f"{i}. {doc.metadata['title']}")
    
    # 混合检索
    ensemble_docs = ensemble_retriever.get_relevant_documents(query)
    print(f"\n🚀 混合检索结果:")
    for i, doc in enumerate(ensemble_docs, 1):
        print(f"{i}. {doc.metadata['title']}")
    
    return {
        "bm25": bm25_docs,
        "semantic": semantic_docs,
        "ensemble": ensemble_docs
    }

# 测试不同类型的查询
test_cases = [
    "深度学习 神经网络 CNN",  # 关键词密集
    "如何理解人工智能的发展趋势？",  # 语义理解
    "机器学习算法有哪些分类？"  # 混合查询
]

for query in test_cases:
    results = compare_retrieval_methods(query)
    print("-" * 60)

# 创建使用混合搜索的RAG系统
hybrid_rag_chain = (
    {"context": ensemble_retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)

print("\n🎯 混合RAG系统测试:")
test_question = "计算机视觉中的CNN技术有什么优势？"

hybrid_answer = hybrid_rag_chain.invoke(test_question)
print(f"问题: {test_question}")
print(f"\n混合RAG回答:\n{hybrid_answer}")

### 5.2 查询重写和扩展

In [ ]:
# 查询重写和扩展
query_rewrite_prompt = ChatPromptTemplate.from_template("""
你是一个查询优化专家。给定一个用户查询，请生成3个不同的重写版本，以提高检索效果。

原始查询: {query}

请生成：
1. 更具体的查询（添加相关术语）
2. 更广泛的查询（使用同义词）
3. 结构化的查询（分解为子问题）

输出格式：
1. [具体查询]
2. [广泛查询] 
3. [结构化查询]
""")

query_rewriter = query_rewrite_prompt | llm | StrOutputParser()

def enhanced_retrieval(original_query: str, k: int = 5):
    """增强检索：使用查询重写"""
    print(f"🔍 增强检索分析")
    print(f"原始查询: {original_query}")
    
    # 生成重写查询
    rewritten_queries = query_rewriter.invoke({"query": original_query})
    print(f"\n📝 重写查询:\n{rewritten_queries}")
    
    # 解析重写的查询
    lines = rewritten_queries.strip().split('\n')
    queries = [original_query]
    for line in lines:
        if line.strip() and not line.startswith('输出格式'):
            # 移除编号前缀
            clean_query = line.split('.', 1)[-1].strip()
            if clean_query and clean_query not in queries:
                queries.append(clean_query)
    
    # 对每个查询进行检索
    all_docs = []
    doc_scores = {}
    
    for i, query in enumerate(queries[:4]):  # 限制查询数量
        print(f"\n检索查询 {i+1}: {query}")
        docs = ensemble_retriever.get_relevant_documents(query)
        
        for doc in docs:
            doc_id = doc.metadata['title']
            if doc_id in doc_scores:
                doc_scores[doc_id] += 1  # 增加权重
            else:
                doc_scores[doc_id] = 1
                all_docs.append(doc)
    
    # 根据得分排序
    sorted_docs = sorted(all_docs, key=lambda x: doc_scores[x.metadata['title']], reverse=True)
    
    print(f"\n📊 综合检索结果 (Top {k}):")
    for i, doc in enumerate(sorted_docs[:k], 1):
        score = doc_scores[doc.metadata['title']]
        print(f"{i}. {doc.metadata['title']} (得分: {score})")
    
    return sorted_docs[:k]

# 测试增强检索
test_query = "AI安全"
enhanced_docs = enhanced_retrieval(test_query, k=3)

# 使用增强检索的结果生成答案
enhanced_context = format_docs(enhanced_docs)
enhanced_answer = rag_chain.invoke(test_query)

print(f"\n🤖 增强RAG回答:")
print(enhanced_answer)

## 6. RAG系统评估

### 6.1 检索质量评估

In [ ]:
# RAG系统评估
def evaluate_rag_system():
    """评估RAG系统的性能"""
    
    # 测试用例：问题和期望相关的文档类别
    test_cases = [
        {
            "question": "什么是卷积神经网络？",
            "expected_categories": ["技术详解", "应用领域"],
            "expected_keywords": ["CNN", "深度学习", "计算机视觉"]
        },
        {
            "question": "大语言模型有哪些发展里程碑？",
            "expected_categories": ["前沿技术"],
            "expected_keywords": ["GPT", "BERT", "LLM"]
        },
        {
            "question": "AI伦理包括哪些问题？",
            "expected_categories": ["伦理安全"],
            "expected_keywords": ["AI伦理", "算法偏见", "隐私保护"]
        }
    ]
    
    print("📊 RAG系统评估")
    print("=" * 50)
    
    total_score = 0
    
    for i, test_case in enumerate(test_cases, 1):
        question = test_case["question"]
        expected_categories = test_case["expected_categories"]
        expected_keywords = test_case["expected_keywords"]
        
        print(f"\n--- 测试用例 {i} ---")
        print(f"问题: {question}")
        
        # 检索文档
        retrieved_docs = ensemble_retriever.get_relevant_documents(question)
        
        # 评估类别匹配
        retrieved_categories = [doc.metadata['category'] for doc in retrieved_docs]
        category_score = len(set(retrieved_categories) & set(expected_categories)) / len(expected_categories)
        
        # 评估关键词匹配
        retrieved_text = " ".join([doc.page_content for doc in retrieved_docs])
        keyword_matches = sum(1 for keyword in expected_keywords if keyword.lower() in retrieved_text.lower())
        keyword_score = keyword_matches / len(expected_keywords)
        
        # 计算综合得分
        overall_score = (category_score + keyword_score) / 2
        total_score += overall_score
        
        print(f"检索到的文档:")
        for j, doc in enumerate(retrieved_docs[:3], 1):
            print(f"  {j}. {doc.metadata['title']} ({doc.metadata['category']})")
        
        print(f"\n评估结果:")
        print(f"  类别匹配度: {category_score:.2f}")
        print(f"  关键词匹配度: {keyword_score:.2f}")
        print(f"  综合得分: {overall_score:.2f}")
    
    average_score = total_score / len(test_cases)
    print(f"\n🎯 整体评估结果:")
    print(f"平均得分: {average_score:.2f}")
    print(f"系统性能: {'🟢 优秀' if average_score >= 0.8 else '🟡 良好' if average_score >= 0.6 else '🔴 需要改进'}")

# 运行评估
evaluate_rag_system()

## 7. 今日学习总结

### ✅ 核心技能掌握

1. **RAG架构理解**
   - RAG解决的核心问题
   - 完整工作流程
   - 组件间的协作机制

2. **向量数据库实践**
   - Chroma：易用性和持久化
   - FAISS：高性能和可扩展性
   - 相似度搜索和评分机制

3. **文档处理技术**
   - 智能文本分块策略
   - 重叠机制保持上下文
   - 元数据管理和利用

4. **检索质量优化**
   - 混合搜索（语义+关键词）
   - 查询重写和扩展
   - 多层检索策略

5. **RAG系统构建**
   - 基础RAG实现
   - 结构化输出和置信度
   - 系统性能评估

### 🎯 关键收获

- **RAG不仅是检索+生成**：需要考虑检索质量、上下文构建、答案生成的整体优化
- **向量数据库选择很重要**：根据数据规模和性能需求选择合适的解决方案
- **检索策略影响最终效果**：混合搜索通常比单一方法效果更好
- **评估体系很关键**：需要建立完善的评估指标来优化系统

### 📝 明日预告：第4天 - LangGraph工作流编排

- StateGraph状态管理
- 复杂工作流设计
- 条件分支和循环
- Human-in-the-loop模式
- 错误处理和重试机制